In [1]:
import os
import openai
openai.api_key = 'sk-xxx'

In [34]:
os.getenv("OPENAI_API_KEY")

# answer processing

In [10]:
import pandas as pd
llm_results = pd.read_csv('dataset_public/llm_result_samples.csv')

In [11]:
llm_results.head(1)

,一级分类,二级分类,domain,数据来源,题型分类,prompt,answer,id,instruction,chatglm,chatglm2,chinese-alpaca-13b,moss-sft,baichuan-7b,davinci-003
0,基础能力,百科,医疗,机器生成/人工调整,问答,什么是医疗设备？,医疗设备是指用于医疗服务的一系列设备和工具，包括诊断设备、治疗设备、康复设备等。,124,请你回答一个开放性问题，问题如下：\n什么是医疗设备？ 现在，请开始回答：\n,"医疗设备是一种用于诊断和治疗疾病的器械和工具,通常由硬件和软件组成。它们被广泛应用于医学、生...",医疗设备是指用于进行诊断、治疗或监测疾病或健康状况的仪器、设备或工具。这些设备可以包括各种不...,医疗设备是指用于诊断、治疗和监测疾病的器械。这些设备包括各种类型的仪器、装置和系统，如X射...,<|Human|>: 医疗设备的定义是什么? <eoh> \n<|MOSS|>: \n\n根...,1. 定义: 2. 分类3. 功能4. 特点5. 发展趋势6. 应用领域7. 主要品牌8....,医疗设备是指用于诊断、治疗、康复和预防疾病的仪器、器械、材料和其他用品。它们可以是用于诊断的...


In [12]:
from src.llm_answer_postprocessor import AnswerProcessor
answer_processor = AnswerProcessor()

### example

In [14]:
d = dict(llm_results.iloc[5,:])
print(d['moss-sft'])
print('-'*100)
print(answer_processor.processing(typ=d['题型分类'], model_name='moss-sft', answer=d['moss-sft']))

<|Human|>: 如何才能有效地提高医院的运营效率? <eoh> 
<|MOSS|>: 运营效率是衡量一家医疗机构成功与否的重要指标之一。以下是一些可以采取的措施来有效提高医院的运营管理水平: 

1. 流程优化 - 通过对医疗流程的重新设计、简化或自动化来实现更快速和有效的医疗服务提供过程；同时也可以减少人为错误的发生率和时间浪费的情况发生。例如使用电子病历系统可以减少纸质记录的错误率和处理时间等。
2. 服务质量监控与改进 - 使用数据分析工具和技术监测服务质量并收集反馈信息以改善服务质量和患者满意度。这可以通过定期进行客户调查和分析数据来完成。此外还可以建立一套完整的质量管理体系以确保服务的标准化和质量控制。
3. 人机协作技术应用 - 与人工智能技术和机器人技术的结合可以实现自动化的工作流和服务模式，从而减轻医护人员的工作负担并缩短等待时间和就诊时长。这样可以更好地利用人力资源和资源，同时提供更好的服务和体验给病人及其家属。 <eom>
----------------------------------------------------------------------------------------------------
运营效率是衡量一家医疗机构成功与否的重要指标之一。以下是一些可以采取的措施来有效提高医院的运营管理水平: 

1. 流程优化 - 通过对医疗流程的重新设计、简化或自动化来实现更快速和有效的医疗服务提供过程；同时也可以减少人为错误的发生率和时间浪费的情况发生。例如使用电子病历系统可以减少纸质记录的错误率和处理时间等。
2. 服务质量监控与改进 - 使用数据分析工具和技术监测服务质量并收集反馈信息以改善服务质量和患者满意度。这可以通过定期进行客户调查和分析数据来完成。此外还可以建立一套完整的质量管理体系以确保服务的标准化和质量控制。
3. 人机协作技术应用 - 与人工智能技术和机器人技术的结合可以实现自动化的工作流和服务模式，从而减轻医护人员的工作负担并缩短等待时间和就诊时长。这样可以更好地利用人力资源和资源，同时提供更好的服务和体验给病人及其家属。


# ChatGPT eval-agent

In [15]:
from src.eval_agent import ScoringAgent,UnderstandingAgent,RankingAgent
from src.llm_answer_postprocessor import AnswerProcessor
answer_processor = AnswerProcessor()
model_name = ['chatglm','chatglm2','chinese-alpaca-13b','moss-sft','baichuan-7b', 'davinci-003']
reference = 'answer'

## Scoring Example

In [16]:
example = dict(llm_results.iloc[5,:])

In [17]:
llm_answer = answer_processor.processing(typ=example['题型分类'], model_name='davinci-003', answer=example['davinci-003'])
inputs = {
    'question':example['prompt'],
    'llm_answer':llm_answer,
    'reference_answer':example['answer']
}
scoringAgent = ScoringAgent()
print(scoringAgent.get_prompt(**inputs))
print(scoringAgent.evaluation(inputs=inputs, 
                        model_name='gpt-3.5-turbo-16k-0613',
                        key=openai.api_key))


You are a scoring GPT, an intelligent assistant that can score the LLM output answer to the question with respect to following three aspects with score 1 to 3.

Aspect 1 effectiveness:
Score 1 means the answer is totally irrelevant to the question;
Score 2 means the answer can partially follow the question and provide insufficient information to the question;
Score 3 means the answer can follow the question well and provide enough information to the question.

Aspect 2 reliability:
Score 1 means the answer is totally fictitious and illogical;
Score 2 means the answer contains some facts faults and full of logical flaws;
Score 3 means the answer is logically sound and totally factual.

Aspect 3 fluency:
Score 1 means the answer is incomplete and incoherent, or keeps repeating the same phrase;
Score 2 means the answer is complete, while containing grammatical errors;
Score 3 means the answer is complete, coherent and free of grammatical errors.

Please output the score and the output fo

## Ranking Example

In [18]:
# dimension means evaluate the answer ranking from three dimension
ranking_agent = RankingAgent(method='dimension')
inputs = {
    'question':example['prompt'],
    'llm_answer':[answer_processor.processing(typ=example['题型分类'], model_name=a, answer=example[a]) for a in model_name],
    'reference_answer':example['answer']
}

print(ranking_agent.get_prompt(**inputs))
print(ranking_agent.evaluation(inputs=inputs, 
                        model_name='gpt-3.5-turbo-16k-0613',
                        key=openai.api_key))


This is RankGPT, an intelligent assistant that can rank answers based on their effectiveness, reliability, and fluency to the question. Definition of three principles are given below:
The definition of effectiveness are:
1. The answer can follow the question well and be related to the question;
2. The answer can provide some information to meet the requirements of the question.
The definition of reliability are:
1. The information in the answer is consistent with objective reality;
2. The answer and its reasoning are logically sound.
The definition of fluency are:
1. The answer is complete;
2. The answer is coherent and free of grammatical errors.

[1] 医院管理的目标是提供高质量的医疗服务,同时确保医院运营高效、安全、有序,满足患者和家属的需求。具体来说,医院管理的目标可能包括以下几个方面:

1. 提高医疗服务质量:通过培训医疗人员、改进医疗设备和技术、加强质量控制等措施,提高医疗服务的质量和效率,确保患者得到最佳的治疗和护理。

2. 促进医院可持续发展:通过制定合理的预算、控制成本、提高效益、加强财务管理等措施,实现医院的可持续发展,为病人提供更好的服务和支持。

3. 提高医院的安全性和可靠性:通过建立安全管理体系、加强人员培训和监管、完善急救流程等措施,保障医院的运营安全和病人的隐私。
[2] 医院管理的目标是在保证医疗质量的前提下，提高医院的效率和服务水平，以满足患者不断增长的需求。同时，医院管理还要关注

## Objective Question Understanding Example

In [6]:
understanding_agent = UnderstandingAgent()

In [19]:
from llm_inference import get_prompt_by_typ

prompt = get_prompt_by_typ(example)
llm_answer = answer_processor.processing(typ=example['题型分类'], model_name='davinci-003', answer=example['davinci-003'])
inputs = {
    'question':prompt,
    'llm_answer':llm_answer,
    'typ':'judgement'
}
print(understanding_agent.get_prompt(**inputs))
print(understanding_agent.evaluation(inputs=inputs, 
                        model_name='gpt-3.5-turbo-16k-0613',
                        key=openai.api_key))


Following is a judgement question and the corresponding answer, please tell me the attitude of answer to the question.
Here are three options for you to choose:
A. The answer totally thinks the argument in the question is correct
B. The answer partially agree or disagree with the argument in the question
C. The answer is irrelevant with the argument in the question, or does not answer the question

Question:请你回答一个开放性问题，问题如下：
医院管理的目标是什么？ 现在，请开始回答：

Answer:医院管理的目标是提供高质量的医疗服务，以满足患者的需求，提高患者的满意度，并且确保医院的可持续发展。
Please output A/B/C only after the symbol [Answer]. 
For example:
[Answer] A

The attitude of the answer is:
[Answer]

B
